#### Download all images from FTP to local filesystem

In [ ]:
import os

from dotenv import load_dotenv
from ftp import SingletonFTP

load_dotenv()
host = os.getenv('FTP_HOST')
username = os.getenv('FTP_USERNAME')
password = os.getenv('FTP_PASSWORD')


ftp = SingletonFTP(host, username, password)
ftp.download_all('/files/files-mailseth', 'tmp')

### extract metadata from filename

In [ ]:
import astral
from astral.sun import sun
from pytz import timezone
location = astral.LocationInfo(
    "Oakland", "USA", "US/Pacific", 37.8044, -122.2712
)

In [ ]:
from datetime import datetime

image_dir = 'tmp'


image_files_list = [{'local_path': os.path.join(image_dir, f)}
               for f in os.listdir(image_dir) if f.endswith('.jpg')]

for output in image_files_list:
    filename = output['local_path'].split('/')[-1]
    camera_name, date, hour, minutes = filename.split('_')[3], filename.split(
        '_')[4], filename.split('_')[7], filename.split('_')[8].split('.')[0]
    seconds = filename.split('_')[8].split('.')[1][0:2]
    year, month, day = date.split('-')
    output['camera_name'] = camera_name
    datetime_obj = datetime(int(year), int(month), int(
        day), int(hour), int(minutes), int(seconds))
    output['datetime'] = datetime_obj
    # Use astral library to get sun information for the specific date and location
    s = sun(location.observer, date=output['datetime'].date(
    ), tzinfo=timezone('US/Pacific'))

    output['S3_key'] = camera_name + '_' + datetime_obj.strftime('%Y%m%d_%H%M%S') + '.jpg'

    # Classify the lighting based on the sun information
    if s['dawn'].time() <= output['datetime'].time() < s['sunrise'].time():
        output['lighting'] = 'dawn'
    elif s['sunrise'].time() <= output['datetime'].time() < s['sunset'].time():
        output['lighting'] = 'day'
    elif s['sunset'].time() <= output['datetime'].time() < s['dusk'].time():
        output['lighting'] = 'dusk'
    else:
        output['lighting'] = 'night'

### Detect objects in frames

In [ ]:
from object_detection import object_detection_runpod
from tqdm import tqdm

from file_helper import write_file_to_s3, get_presigned_url


image_files_iter = iter(image_files_list)

output_list = []

image_files_in_output_list = [output['image_file'] for output in output_list]

for image_file in tqdm(image_files_iter):
    if image_file not in image_files_in_output_list:
        write_file_to_s3(image_file['S3_key'], image_file['local_path'])
        image_url = get_presigned_url(image_file['S3_key'])

        detection_result = object_detection_runpod(image_url)
        detection_result_json = detection_result.pandas().xyxy[0].to_json()
        print('image_file', image_file)
        print('detection_result_json', detection_result_json)
        output_list.append({
            'image_file': image_file,
            'detection_result': detection_result_json
        })
        # Explicitly delete the detection_result and call the garbage collector
        del detection_result